# Проектирование интерфейса

Определение структуры приложения и его функциональности, создание макета интерфейса приложения

Пример структуры данных
- Имя
- Дата рождения
- Город
- Желаемая должность
- Контакты
  - Телефон
  - email
  - telegram
- О себе
- Навыки
- Образование
  - Тип образования (высшее, дополнительное, курсы)
  - Дата начала
  - Дата окончания
  - Учебное заведение
  - Факультет, специализация
- Опыт работы
  - Тип (основная деятельность, фриланс)
  - Дата начала
  - Дата окончания
  - Организация
  - Город
  - Сайт
  - Должность
  - Деятельность
- Созданные НС
- Публикации

# Создание базы данных

Пример структуры таблиц БД
```
[users] - пользователи
id (int) - id пользователя
username (str) - имя пользователя
password (str) - пароль
```

```
[positions] - должности
id (int) - id должности
position (str) - должность
sort (int) - сортировка
```

```
[user_skills] - навыки пользователя
id (int) - id записи
user_id (users.id) - id пользователя
skill (str) - название навыка
sort (int) - сортировка
```

```
[education_type] - тип образования (высшее, дополнительное, курсы)
id (int) - id записи
education_type (str) - тип образования
sort (int) - сортировка
```

```
[experience_type] - тип опыта работы (основная деятельность, фриланс)
id (int) - id записи
experience_type (str) - тип опыта работы
sort (int) - сортировка
```

```
[resume] - резюме пользователя
id (int) - id резюме
user_id (users.id) - id пользователя, внешний ключ
resume_name (str) - название резюме
name (str) - имя пользователя
city (str) - город
position_id (positions.id) - id должности, внешний ключ
custom_position (str) - другая должность
phone (str) - основной телефон
email (str) - основной email
github (str) - github
about (str) - о себе
```

```
[user_contacts] - контакты пользователя
id (int) - id записи
resume_id (resume.id) - резюме пользователя, внешний ключ
contact_name (str) - название контакта (linkedin, youtube, whatsapp, ...)
contact_val (str) - контакт
sort (int) - сортировка
```

```
[education] - образование пользователя
id (int) - id записи
resume_id (resume.id) - резюме пользователя, внешний ключ
education_type_id (education_type.id) - тип образования, внешний ключ
date_start (datetime) - дата поступления
date_end (datetime) - дата окончания
institution - учебное заведение
faculty - факультет, специализация
sort (int) - сортировка
```

```
[experience] - опыт работы пользователя
id (int) - id записи
resume_id (resume.id) - резюме пользователя, внешний ключ
experience_type_id (experience_type) - тип опыта работы, внешний ключ
date_start (datetime) - дата начала работы
date_end (datetime) - дата окончания работы
organisation (str) - организация
city (str) - город
web (str) - web-страница
position (str) - должность
activity (str) - деятельность
sort (int) - сортировка
```

Примеры запросов в базу данных
```
-- Получение списка резюме пользователя
SELECT id, resume_name
FROM resume
WHERE user_id = @user_id
```

```
-- Получение списка должностей
SELECT id, position
FROM positions
ORDER BY sort
```

```
-- Получение списка навыков пользователя
SELECT id, skill
FROM skills
WHERE user_id = @user_id
ORDER BY sort
```

```
-- Получение резюме пользователя
SELECT resume.id, resume.resume_name, resume.name, resume.city,
    positions.position, resume.custom_position, resume.phone,
    resume.email, resume.github, resume.about
FROM resume
    LEFT JOIN positions ON resume.position_id = positions.id
WHERE user_id = @user_id
```

```
-- Получение образования пользователя
SELECT education.id, education_type.education_type,
    education.date_start, education.date_end,
    education.institution, education.faculty
FROM education
    INNER JOIN education_type ON education.education_type_id = education_type.id
WHERE resume_id = @resume_id
ORDER BY education_type.sort, education.sort
```

```
-- Получение опыта работы пользователя
SELECT experience.id, experience_type_id.experience_type,
    experience.date_start, experience.date_end, experience.organisation,
    experience.city, experience.web, experience.position, experience.activity
FROM experience
    INNER JOIN experience_type ON experience.experience_type_id = experience_type.id
WHERE resume_id = @resume_id
ORDER BY experience_type.sort, experience.sort
```

# Серверная часть (backend)

Обработка запросов от клиентской части (frontend), взаимодействие с базой данных (CRUD)

Примеры моделей
```
class Resume(Base):
    __tablename__ = 'resume'
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey(User.id))
    resume_name = Column(String(30))  # название резюме

    # личные данные
    name = Column(String(100))  # ФИО
    birthdate = Column(DateTime)  # дата рождения
    city = Column(String(50))  # город проживания

    # основные контакты
    phone = Column(String(30))   # основной телефон
    email = Column(String(50))   # основной email
    github = Column(String(50))  # основной github

    # должность претендента
    position_id = Column(Integer, ForeignKey(Positions.id))  # желаемая должность
    custom_position = Column(String(50))  # другая должность

    # о себе
    about = Column(String)

    # навыки (основная таблица с навыками - AppFormSkills)
    custom_skills = Column(String)  # прочие навыки
```

```
class Education(Base):
    __tablename__ = 'education'
    id = Column(Integer, primary_key=True)
    resume_id = Column(Integer, ForeignKey(AppForm.id))
    education_type_id = Column(Integer, ForeignKey(EducationType.id))
    date_start = Column(String(7), default=None)  # дата начала обучения
    date_end = Column(String(7), default=None)  # дата окончания обучения
    institution = Column(String(150), default=None)  # учебное заведение
    faculty = Column(String(150), default=None)  # факультет, специализация
    sort = Column(Integer)
```


Примеры ORM
```
# получение резюме пользователя
get_user_resume(user_id):

    with SessionLocal() as session:
        resume = session.query(Resume).filter_by(id=resume_id).first()

        return vars(resume)
```

```
# получение типов образования
get_education_type(user_id):

    with (SessionLocal() as session):
        education_type = session.query(EducationType).order_by(EducationType.sort)

        return education_type.all()
```

```
# получение образования пользователя
get_user_education(user_id, resume_id):

    with (SessionLocal() as session):
        query = session.query(
            Education.id,
            EducationType.education_type,
            Education.date_start,
            Education.date_end,
            Education.institution,
            Education.faculty
        ).join(
            EducationType,
            Education.education_type_id == EducationType.id
        ).join(
            Resume,
            Education.resume_id == Resume.id
        ).filter(
            Education.resume_id == resume_id,
            AppForm.user_id == user_id
        ).order_by(
            EducationType.sort,
            Education.sort
        ).all()

        return query
```

Передача данных в форму
```
@app.get('/resume/{resume_id}')
async def resume_appform(request: Request, resume_id: int):

    user_resume_list = get_user_resume_list(user_id)  # список резюме пользователя
    user_resume = get_user_resume(resume_id)  # текущая анкета пользователя
    user_positions = get_user_positions()  # список должностей
    user_skills = get_user_skills(resume_id)  # список навыков пользователя
    user_education = get_user_education(user_id, resume_id)  # образование пользователя
    user_experience = get_user_experience(user_id, resume_id)  # опыт работы пользователя
    
    context = {'request': request, 'resume_id': resume_id,
               'user_resume_list': user_resume_list,
               'user_resume': user_resume,
               'user_positions': user_positions,
               'user_skills': user_skills,
               'user_education': user_education,
               'user_experience': user_experience}

    return templates.TemplateResponse('resume.html', context=context)
```

# Клиентская часть (frontend)

Используется шаблонизатор **Jinja2**

<div class="row p-2">
    <div class="col gx-3">
        <label for="name" class="form-label">ФИО</label>
        <input class="form-control form-control-sm" type="text" id="name" name="name" value="{{ user_resume.name }}">
    </div>
</div>
<div class="row p-2">
    <div class="col-md gx-3">
        <label for="city" class="form-label">Город</label>
        <input class="form-control form-control-sm" type="text" id="city" name="city" value="{{ user_resume.city }}">
    </div>
    <div class="col-md gx-3">
        <label for="birthdate" class="form-label">Дата рождения</label>
        <input class="form-control form-control-sm" type="text" id="birthdate" name="birthdate" value="{{ user_resume.birthdate }}">
    </div>
    <div class="col-md gx-3">
        <label for="phone" class="form-label">Телефон</label>
        <input class="form-control form-control-sm" type="text" id="phone" name="phone" value="{{ user_resume.phone }}">
    </div>
</div>
<div class="row p-2">
    <div class="col-md gx-3">
        <label for="email" class="form-label">email</label>
        <input class="form-control form-control-sm" type="text" id="email" name="email" value="{{ user_resume.email }}">
    </div>
    <div class="col-md gx-3">
        <label for="github" class="form-label">Github</label>
        <input class="form-control form-control-sm" type="text" id="github" name="github" value="{{ user_resume.github }}">
    </div>
</div>

```
<div class="row p-2">
    <div class="col gx-3">
        <label for="name" class="form-label">ФИО</label>
        <input class="form-control form-control-sm" type="text" id="name" name="name" value="{% if user_resume.name %}{{ user_resume.name }}{% endif %}">
    </div>
</div>
<div class="row p-2">
    <div class="col-md gx-3">
        <label for="city" class="form-label">Город</label>
        <input class="form-control form-control-sm" type="text" id="city" name="city" value="{% if user_resume.city %}{{ user_resume.city }}{% endif %}">
    </div>
    <div class="col-md gx-3">
        <label for="birthdate" class="form-label">Дата рождения</label>
        <input class="form-control form-control-sm" type="text" id="birthdate" name="birthdate" value="{% if user_resume.birthdate %}{{ user_resume.birthdate }}{% endif %}">
    </div>
    <div class="col-md gx-3">
        <label for="phone" class="form-label">Телефон</label>
        <input class="form-control form-control-sm" type="text" id="phone" name="phone" value="{% if user_resume.phone %}{{ user_resume.phone }}{% endif %}">
    </div>
</div>
<div class="row p-2">
    <div class="col-md gx-3">
        <label for="email" class="form-label">email</label>
        <input class="form-control form-control-sm" type="text" id="email" name="email" value="{% if user_resume.email %}{{ user_resume.email }}{% endif %}">
    </div>
    <div class="col-md gx-3">
        <label for="github" class="form-label">Github</label>
        <input class="form-control form-control-sm" type="text" id="github" name="github" value="{% if user_resume.github %}{{ user_resume.github }}{% endif %}">
    </div>
</div>
```

```
{% if user_skills %}
<div class="container">
    <p>Укажите ваши навыки</p>
    {% for items in user_skills %}
    <div class="form-check form-check-inline">
        <input class="form-check-input" type="checkbox" value="{% if items.2 %}1{% endif %}" id="skill{{ items.0 }}" name="skill{{ items.0 }}"{% if items.2 %} checked{% endif %}>
        <label class="form-check-label" for="skill{{ items.1 }}">{{ items.1 }}</label>
    </div>
    {% endfor %}
</div>
{% endif %}
```